# Insights from ESG Indicator - A SQL Project
### *by Lan Hoang*

## I. Introduction

This SQL project aims to put in practice and demonstrate my SQL skills, through obtaining data-driven insights from ESG (Environment/Social/Governance) indicators. These insights are intended to help shareholders and investors better understand how these ESG indicators evolve over time and compare companies in the same sector for benchmarking.

This project uses a dummy data set obtained from Tableau Exchange at the following link: https://exchange.tableau.com/fr-FR/products/625. 

The SQL queries in this project would answer the following key business questions: 

- Where are we on environmental/social issues/governance?
- Where are we in terms of CO2 emissions?
- How many companies have at least one social policy?
- What is the percentage of women on the board of directors?
- How is my company performing compared to companies in the same industry?

Attributes in the data set:
- Date (Fiscal year)
- Company (Company name)
- Business sector (Business sector Type 1 (Communications, Consumer discretionary, Energy...))
- Sector of activity 2 Sector of activity Type 2 (Multimedia, Telecommunications, Retail...)
- Sector of activity 3 (Sector of activity Type 3 (Advertising, Internet, Publishing))
- Sector of activity 4 (Sector of activity Type 4 (Cinema/TV, Music, Furniture...))
- Country (Country of the company)
- Human Rights Policy ("Y" if there is a policy)
- Equal Opportunity Policy ("Y" if there is a policy)
- Fair remuneration policy ("Y" if there is a policy)
- Health security policy ("Y" if there is a policy)
- Climate Change Policy ("Y" if there is a policy)
- Energy efficiency policy ("Y" if there is a policy)
- Waste reduction policy ("Y" if there is a policy)
- Water policy ("Y" if there is a policy)
- Biodiversity policy ("Y" if there is a policy)
- ESG Score (ESG Diffusion Score (0.1 to 100))
- CO2 emissions (Greenhouse gases - Estimated CO2 (in K tonnes))
- CO2 per sales (Greenhouse gas - CO2 per sales (estimate in tonnes per sales M€))
- Direct CO2 emissions (Greenhouse gases - Estimated direct CO2 emissions (in K tonnes))
- Indirect CO2 emissions (Greenhouse gases - Estimated indirect CO2 emissions (in K tonnes))
- Energy consumption (Energy consumption (in MWh))
- Waste (Total weight of waste the company disposes of (in K tons))
- % of women in management (Percentage of women in senior management positions in the company (0.5 represents 50%))
- % of women on the board of directors (Percentage of women on the board of directors (0.5 represents 50%))
- Market Cap (Company Market Cap Amount ($M))
- Number of employees (Number of employees)

## II. Data Import and Setting Up

In this step, I use Python to import the data set using Pandas. I then create a Python class to help create a SQL table for later upload onto SQLite database.

In [1]:
import sqlite3
import csv
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
esg_data = pd.read_csv("ESG Data Extract_Extract.csv", sep = ",")

esg_data.head()

,Biodiversity Policy,Climate Change Policy,Company,Company Id,Country,Date,Emissions Reduction Initiatives,Energy Efficiency Policy,Equal Opportunity Policy,Fair Remuneration Policy,...,Energy Consumption,ESG Score,Indirect CO2 Emissions,Market Cap,Nb Fatalities,Nb of Employees,Pct Women in Management,Pct Women on Board,Row Id,Waste
0,Y,Y,Company 1,1,Indonesia,01-Jan-14,N,Y,N,Y,...,NaN,25.57,0.026,17574.4439,NaN,259.331,NaN,0.098607,10890,NaN
1,Y,N,Company 1,1,Indonesia,01-Jan-20,N,N,N,N,...,2.22,42.50,0.141,17232.5546,NaN,221.165,0.1326,0.326082,10884,0.105
2,Y,N,Company 1,1,Indonesia,01-Jan-19,N,N,N,N,...,2.38,45.79,0.140,18613.9051,NaN,253.592,0.2745,0.317088,10885,0.107
3,Y,N,Company 1,1,Indonesia,01-Jan-18,N,N,N,N,...,NaN,35.70,0.156,14351.5871,NaN,250.795,0.2383,0.221907,10886,NaN
4,Y,Y,Company 1,1,Indonesia,01-Jan-16,N,N,N,N,...,NaN,35.15,0.269,16668.9627,NaN,264.743,0.1927,0.294612,10888,NaN


In [3]:
esg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10890 entries, 0 to 10889
Data columns (total 36 columns):
Biodiversity Policy                  10890 non-null object
Climate Change Policy                10890 non-null object
Company                              10890 non-null object
Company Id                           10890 non-null int64
Country                              10890 non-null object
Date                                 10890 non-null object
Emissions Reduction Initiatives      10890 non-null object
Energy Efficiency Policy             10890 non-null object
Equal Opportunity Policy             10890 non-null object
Fair Remuneration Policy             10890 non-null object
Fiscal Year                          10890 non-null int64
Gender Pay Gap Breakout              10890 non-null object
Health Safety Policy                 10890 non-null object
Human Rights Policy                  10890 non-null object
Industry                             10890 non-null object
Industry

In [4]:
esg_data.columns = esg_data.columns.str.replace(' ', '_')
#source: https://stackoverflow.com/questions/41476150/remove-or-replace-spaces-in-column-names

In [5]:
class SQLCommandGen:
    def __init__(self, col_names, col_dtypes): 
        # create an attribute 
        self.col_names = col_names
        self.col_dtypes = col_dtypes
        
    @classmethod
    def SQL_creat_tab(cls, col_names, col_dtypes):
        #working on column names
        col_w_underscore = list(col_names.str.replace(' ', '_'))
        
        #Defining data types
        dtypes_list = []
        for item in list(col_dtypes):
            if item == np.object:
                dtypes_list.append("text")
            elif item == np.int64:
                dtypes_list.append("int")
            elif item == np.float64:
                dtypes_list.append("float")
        tab_name_zip = list(zip(col_w_underscore, dtypes_list))
        
        #bring the two lists together
        tab_name_list = []
        for tuple in tab_name_zip:
            tab_name_list.append(str(tuple))
        
        #create the final string after stripping all the brackets and quotes - could also conver this into a function
        tab_name1 = ''
        for string in tab_name_list:
            string = string.strip("()")
            string = string.replace(",", "")
            string = string.replace("'", "")
            tab_name1 = tab_name1 + ", " + string
            tab_name1 = tab_name1.strip(", ") #to get rid of the first ", "
            
        #Now create the table command
        table_command = '''CREATE TABLE IF NOT EXISTS esg_data (''' + tab_name1 + ''');'''
        return table_command
        

In [6]:
esg_table_SQL_command = SQLCommandGen.SQL_creat_tab(esg_data.columns, esg_data.dtypes)
esg_table_SQL_command

'CREATE TABLE IF NOT EXISTS esg_data (Biodiversity_Policy text, Climate_Change_Policy text, Company text, Company_Id int, Country text, Date text, Emissions_Reduction_Initiatives text, Energy_Efficiency_Policy text, Equal_Opportunity_Policy text, Fair_Remuneration_Policy text, Fiscal_Year int, Gender_Pay_Gap_Breakout text, Health_Safety_Policy text, Human_Rights_Policy text, Industry text, Industry_2 text, Industry_3 text, Industry_4 text, Nb_of_Environmental_Fines text, Risks_of_Climate_Change_Discussed text, Waste_Reduction_Policy text, Water_Policy text, Years_Auditor_Employed text, CO2_Emissions float, CO2_per_Sales float, Direct_CO2_Emissions float, Energy_Consumption float, ESG_Score float, Indirect_CO2_Emissions float, Market_Cap float, Nb_Fatalities float, Nb_of_Employees float, Pct_Women_in_Management float, Pct_Women_on_Board float, Row_Id int, Waste float);'

In [7]:
#Create empty database
Path("esg.db").touch()

#Connect to database
connection = sqlite3.connect("esg.db")
cursor = connection.cursor()

#Create a table
cursor.execute(esg_table_SQL_command)

In [8]:
#add table to database
esg_data.to_sql("esg_data", connection, if_exists="replace", index=False)

In [9]:
#trying to get the first row for inspection
cursor.execute("""SELECT * FROM esg_data""").fetchone()


('Y',
 'Y',
 'Company 1',
 1,
 'Indonesia',
 '01-Jan-14',
 'N',
 'Y',
 'N',
 'Y',
 2014,
 'N.A.',
 'N',
 'N',
 'Financials',
 'Financial Services',
 'Asset Management',
 'Private Equity',
 'N.A.',
 'Y',
 'Y',
 'Y',
 'N.A.',
 0.057,
 0.316,
 0.031,
 None,
 25.57,
 0.026000000000000002,
 17574.4439,
 None,
 259.33099999999996,
 None,
 0.09860707199999999,
 10890,
 None)

In [10]:
#convert query results into a dataframe
pd.read_sql_query("""SELECT * FROM esg_data LIMIT 5""", con = connection)


,Biodiversity_Policy,Climate_Change_Policy,Company,Company_Id,Country,Date,Emissions_Reduction_Initiatives,Energy_Efficiency_Policy,Equal_Opportunity_Policy,Fair_Remuneration_Policy,...,Energy_Consumption,ESG_Score,Indirect_CO2_Emissions,Market_Cap,Nb_Fatalities,Nb_of_Employees,Pct_Women_in_Management,Pct_Women_on_Board,Row_Id,Waste
0,Y,Y,Company 1,1,Indonesia,01-Jan-14,N,Y,N,Y,...,NaN,25.57,0.026,17574.4439,None,259.331,NaN,0.098607,10890,NaN
1,Y,N,Company 1,1,Indonesia,01-Jan-20,N,N,N,N,...,2.22,42.50,0.141,17232.5546,None,221.165,0.1326,0.326082,10884,0.105
2,Y,N,Company 1,1,Indonesia,01-Jan-19,N,N,N,N,...,2.38,45.79,0.140,18613.9051,None,253.592,0.2745,0.317088,10885,0.107
3,Y,N,Company 1,1,Indonesia,01-Jan-18,N,N,N,N,...,NaN,35.70,0.156,14351.5871,None,250.795,0.2383,0.221907,10886,NaN
4,Y,Y,Company 1,1,Indonesia,01-Jan-16,N,N,N,N,...,NaN,35.15,0.269,16668.9627,None,264.743,0.1927,0.294612,10888,NaN


## III. Data retrieval and analysis using SQL

We want to do some exploratory data analysis with our data set. How many countries are there in the data set?
How many companies? How many companies per country?

In [11]:
qry = """SELECT COUNT (DISTINCT Country) FROM esg_data;"""

pd.read_sql_query(qry, con = connection)

,COUNT (DISTINCT Country)
0,34


In [12]:
qry = """SELECT COUNT (DISTINCT Company) FROM esg_data;"""

pd.read_sql_query(qry, con = connection)

,COUNT (DISTINCT Company)
0,1656


In [13]:
qry = """
SELECT Country, COUNT (DISTINCT Company) AS No_Comp 
FROM esg_data 
GROUP BY Country 
ORDER BY No_Comp DESC;"""

pd.read_sql_query(qry, con = connection)

,Country,No_Comp
0,Argentina,683
1,Ecuador,290
2,Indonesia,89
3,Hungary,69
4,Japan,61
5,France,59
6,United Kingdom,56
7,Australia,42
8,Austria,38
9,Ethiopia,34


Argentina seems to have the most companies represented in this data set. And we have quite a few countries with very few companies represented (with less than 10 companies). 

Next up, we investigate the timeline of the data. Which years does this data set cover?

In [14]:
qry = """
SELECT DISTINCT Fiscal_Year 
FROM esg_data 
ORDER BY Fiscal_Year DESC;"""

pd.read_sql_query(qry, con = connection)

,Fiscal_Year
0,2020
1,2019
2,2018
3,2017
4,2016
5,2015
6,2014


The data set covers 7 years of historical data, but only up to 2020. 

Now we look at the average global ESG scores for each year to discern a trend:

In [15]:
qry = """
SELECT Fiscal_Year, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score 
FROM esg_data 
GROUP BY Fiscal_Year 
ORDER BY Avg_ESG_Score DESC;"""

pd.read_sql_query(qry, con = connection)

,Fiscal_Year,Avg_ESG_Score
0,2020,37.9
1,2019,36.8
2,2018,35.4
3,2017,34.1
4,2016,32.5
5,2015,31.2
6,2014,30.1


Looks like the trend of global ESG score in this toy data set is conveniently set to improve over the years. That might also make sense in real life given that the world and its citizens are getting more aware and conscious of the sustainability.

Now we want to look at the countries with the highest and lowest ESG scores in 2020.

In [16]:
qry = """
SELECT Country, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
WHERE Fiscal_Year = 2020 
GROUP BY Country
ORDER BY Avg_ESG_Score DESC
LIMIT 10;"""

pd.read_sql_query(qry, con = connection)

,Country,Avg_ESG_Score
0,Bulgaria,53.0
1,Canada,52.5
2,Costa Rica,49.9
3,Georgia,47.7
4,Italy,47.6
5,Estonia,46.7
6,United Kingdom,46.4
7,Croatia,44.7
8,Austria,44.6
9,France,43.5


In [17]:
qry = """
SELECT Country, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
WHERE Fiscal_Year = 2020 
GROUP BY Country
ORDER BY Avg_ESG_Score ASC
LIMIT 10;"""

pd.read_sql_query(qry, con = connection)

,Country,Avg_ESG_Score
0,Israel,25.0
1,Egypt,25.6
2,Denmark,27.6
3,China,28.3
4,Argentina,31.6
5,Germany,32.3
6,Spain,35.0
7,Brazil,36.7
8,Czech Republic,37.1
9,Ecuador,37.5


Now we move on to look at the ESG scores by industries. Overall, which are the top 3 and bottom 3 industries in terms of ESG scores? In 2020? 

In [18]:
qry = """
SELECT Industry, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data  
GROUP BY Industry
ORDER BY Avg_ESG_Score DESC
LIMIT 3;"""

pd.read_sql_query(qry, con = connection)

,Industry,Avg_ESG_Score
0,Materials,42.9
1,Energy,42.1
2,Utilities,42.0


In [19]:
qry = """
SELECT Industry, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data  
GROUP BY Industry
ORDER BY Avg_ESG_Score ASC
LIMIT 3;"""

pd.read_sql_query(qry, con = connection)

,Industry,Avg_ESG_Score
0,Communications,27.9
1,Technology,28.8
2,Real Estate,29.8


In [20]:
qry = """
SELECT Industry, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
WHERE Fiscal_Year = 2020
GROUP BY Industry
ORDER BY Avg_ESG_Score DESC
LIMIT 3;"""

pd.read_sql_query(qry, con = connection)

,Industry,Avg_ESG_Score
0,Energy,48.7
1,Materials,45.7
2,Utilities,44.7


In [21]:
qry = """
SELECT Industry, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
WHERE Fiscal_Year = 2020
GROUP BY Industry
ORDER BY Avg_ESG_Score ASC
LIMIT 3;"""

pd.read_sql_query(qry, con = connection)

,Industry,Avg_ESG_Score
0,Communications,30.5
1,Technology,33.2
2,Consumer Discretionary,36.2


Interestingly the Energy industry seems to have move up in 2020 to become the industry with the highest ESG score of 48.7.

Now we look at the companies, which are the top 10 and bottom 10 companies in terms of ESG score, which industry do they belong to? Over 7 years and in 2020?

In [22]:
qry = """
SELECT Company, Industry, Industry_2, Industry_3, Industry_4, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
GROUP BY Company
ORDER BY Avg_ESG_Score DESC
LIMIT 5;"""

pd.read_sql_query(qry, con = connection)

,Company,Industry,Industry_2,Industry_3,Industry_4,Avg_ESG_Score
0,Company 665,Energy,Oil & Gas,Oil & Gas Producers,Exploration & Production,68.2
1,Company 1020,Materials,Materials,Metals & Mining,Precious Metals,67.6
2,Company 992,Materials,Materials,Chemicals,Agricultural Chemicals,66.4
3,Company 323,Materials,Materials,Construction Materials,Building Materials,66.1
4,Company 226,Energy,Oil & Gas,Oil & Gas Producers,Integrated Oils,64.7


In [23]:
qry = """
SELECT Company, Industry, Industry_2, Industry_3, Industry_4, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
GROUP BY Company
ORDER BY Avg_ESG_Score ASC
LIMIT 5;"""

pd.read_sql_query(qry, con = connection)

,Company,Industry,Industry_2,Industry_3,Industry_4,Avg_ESG_Score
0,Company 615,Industrials,Industrial Services,Commercial Support Services,Waste Management,NaN
1,Company 756,Energy,Oil & Gas,Oil & Gas Producers,Refining & Marketing,6.1
2,Company 1258,Technology,Software & Tech Services,Software,Application Software,7.8
3,Company 407,Real Estate,Real Estate,REIT,Residential REIT,7.9
4,Company 1370,Technology,Software & Tech Services,Technology Services,Data & Transaction Processors,8.0


Interestingly, we also find Energy companies in the bottom rank in terms of ESG scores, not surprising given they are in the Oil & Gas industry which is still in the middle of transitioning to green energy.

Now in 2020 only.

In [24]:
qry = """
SELECT Company, Industry, Industry_2, Industry_3, Industry_4, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
WHERE Fiscal_Year = 2020
GROUP BY Company
ORDER BY Avg_ESG_Score DESC
LIMIT 5;"""

pd.read_sql_query(qry, con = connection)

,Company,Industry,Industry_2,Industry_3,Industry_4,Avg_ESG_Score
0,Company 496,Utilities,Utilities,Electric Utilities,Power Generation,67.9
1,Company 984,Materials,Materials,Containers & Packaging,Containers & Packaging,67.6
2,Company 617,Materials,Materials,Chemicals,Specialty Chemicals,66.6
3,Company 1549,Utilities,Utilities,Electric Utilities,Power Generation,65.7
4,Company 226,Energy,Oil & Gas,Oil & Gas Producers,Integrated Oils,65.7


In [25]:
qry = """
SELECT Company, Industry, Industry_2, Industry_3, Industry_4, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
WHERE Fiscal_Year = 2020
GROUP BY Company
ORDER BY Avg_ESG_Score ASC
LIMIT 5;"""

pd.read_sql_query(qry, con = connection)

,Company,Industry,Industry_2,Industry_3,Industry_4,Avg_ESG_Score
0,Company 100,Materials,Materials,Metals & Mining,Base Metals,None
1,Company 1002,Industrials,Industrial Services,Transportation & Logistics,Transit Services,None
2,Company 1013,Consumer Staples,Consumer Staple Products,Food,Packaged Food,None
3,Company 1020,Materials,Materials,Metals & Mining,Precious Metals,None
4,Company 1022,Technology,Software & Tech Services,Technology Services,Data & Transaction Processors,None


Apparently some companies don't have ESG scores, will need to investigate the null values. First we start with the ESG_Score column

In [26]:
qry = """
SELECT Company, Fiscal_Year, ESG_Score
FROM esg_data 
WHERE ESG_Score IS NULL;"""

pd.read_sql_query(qry, con = connection)

,Company,Fiscal_Year,ESG_Score
0,Company 7,2020,None
1,Company 11,2020,None
2,Company 12,2020,None
3,Company 31,2020,None
4,Company 33,2020,None
5,Company 40,2020,None
6,Company 50,2020,None
7,Company 52,2020,None
8,Company 72,2016,None
9,Company 80,2020,None


Now trying to count the number of companies that have one or more missing ESG score.

In [27]:
qry = """
SELECT COUNT(DISTINCT Company) AS No_Missing_Year
FROM esg_data 
WHERE ESG_Score IS NULL;"""

pd.read_sql_query(qry, con = connection)

,No_Missing_Year
0,234


We have quite a few companies over the years that do not have ESG scores (249 companies), let's see now by the count of years by company.

In [28]:
qry = """
SELECT Company, COUNT(Fiscal_Year) AS No_Year, ESG_Score
FROM esg_data 
WHERE ESG_Score IS NULL
GROUP BY Company
ORDER BY No_Year DESC;"""

pd.read_sql_query(qry, con = connection)

,Company,No_Year,ESG_Score
0,Company 269,5,None
1,Company 1526,3,None
2,Company 1122,2,None
3,Company 1189,2,None
4,Company 1225,2,None
5,Company 125,2,None
6,Company 1499,2,None
7,Company 363,2,None
8,Company 483,2,None
9,Company 877,2,None


So most companies with missing ESG scores only miss 1 year, only 10 companies miss 2 or more years. Company 269 seems to be the one that miss the most (5 years).

To make the calculation of average ESG scores more accurate, we need to remove the records with ESG scores out of the calculation. First we start with top companies then bottom companies.

In [29]:
qry = """
SELECT Company, Industry, Industry_2, Industry_3, Industry_4, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
WHERE Fiscal_Year = 2020 AND ESG_Score IS NOT NULL
GROUP BY Company
ORDER BY Avg_ESG_Score DESC
LIMIT 5;"""

pd.read_sql_query(qry, con = connection)

,Company,Industry,Industry_2,Industry_3,Industry_4,Avg_ESG_Score
0,Company 496,Utilities,Utilities,Electric Utilities,Power Generation,67.9
1,Company 984,Materials,Materials,Containers & Packaging,Containers & Packaging,67.6
2,Company 617,Materials,Materials,Chemicals,Specialty Chemicals,66.6
3,Company 1549,Utilities,Utilities,Electric Utilities,Power Generation,65.7
4,Company 226,Energy,Oil & Gas,Oil & Gas Producers,Integrated Oils,65.7


In [30]:
qry = """
SELECT Company, Industry, Industry_2, Industry_3, Industry_4, ROUND(AVG(ESG_Score),1) AS Avg_ESG_Score FROM esg_data 
WHERE Fiscal_Year = 2020 AND ESG_Score IS NOT NULL
GROUP BY Company
ORDER BY Avg_ESG_Score ASC
LIMIT 5;"""

pd.read_sql_query(qry, con = connection)

,Company,Industry,Industry_2,Industry_3,Industry_4,Avg_ESG_Score
0,Company 46,Materials,Materials,Chemicals,Basic & Diversified Chemicals,0.8
1,Company 230,Consumer Discretionary,Consumer Discretionary Products,Automotive,Auto Parts,4.2
2,Company 1023,Communications,Media,Entertainment Content,Video Games,8.9
3,Company 305,Technology,Software & Tech Services,Software,Infrastructure Software,9.0
4,Company 902,Communications,Media,Internet Media & Services,Internet Media & Services,9.9


It doesn't seem to change the calculation of the top companies, but it sure does improve the calculation of the bottom companies.

### Social Policy

Now we want to see how many companies have at least 1 social policy. In this data set, a social policies are defined to include Health Safety policy, Human Rights policy, Equal Opportunity policy and Fair Remuneration policy. 

In [31]:
qry = """
SELECT COUNT(DISTINCT Company) AS No_Comp_w_Social_Policy
FROM esg_data 
WHERE Health_Safety_Policy = "Y" OR Human_Rights_Policy = "Y" OR Equal_Opportunity_Policy = "Y" OR Fair_Remuneration_Policy = "Y";"""

pd.read_sql_query(qry, con = connection)

,No_Comp_w_Social_Policy
0,1620


What is the percentage out of all the companies we have?

In [32]:
qry = """
SELECT COUNT(DISTINCT Company) * 100 / (SELECT COUNT(DISTINCT Company) FROM esg_data) AS pct_comp_w_Social_Policy
FROM esg_data 
WHERE Health_Safety_Policy = "Y" OR Human_Rights_Policy = "Y" OR Equal_Opportunity_Policy = "Y" OR Fair_Remuneration_Policy = "Y";"""

pd.read_sql_query(qry, con = connection)

,pct_comp_w_Social_Policy
0,97


Not a bad number, we have 97% coverage of social policy. Let's dig a little deeper and now we see how many companies by the number of policies they have: 

### Share of Women on the Boards of companies

In [34]:
qry = """    
SELECT Company, Pct_Women_in_Management, Pct_Women_on_Board
FROM esg_data
WHERE Fiscal_Year = 2020
ORDER BY Pct_Women_in_Management DESC, Pct_Women_on_Board DESC
LIMIT 10;
"""

pd.read_sql_query(qry, con = connection)

,Company,Pct_Women_in_Management,Pct_Women_on_Board
0,Company 597,0.5744,0.275263
1,Company 39,0.5358,0.293835
2,Company 179,0.5277,0.044864
3,Company 521,0.5006,0.384632
4,Company 513,0.4928,0.299137
5,Company 1192,0.4853,0.400138
6,Company 601,0.4789,0.279494
7,Company 1188,0.4666,0.179928
8,Company 313,0.4637,0.324326
9,Company 449,0.4571,0.222636


In [36]:
qry = """    
SELECT Company, Pct_Women_in_Management, Pct_Women_on_Board
FROM esg_data
WHERE Fiscal_Year = 2020
ORDER BY Pct_Women_in_Management ASC, Pct_Women_on_Board ASC
LIMIT 10;
"""

pd.read_sql_query(qry, con = connection)

,Company,Pct_Women_in_Management,Pct_Women_on_Board
0,Company 1306,None,NaN
1,Company 8,None,0.0
2,Company 16,None,0.0
3,Company 74,None,0.0
4,Company 772,None,0.0
5,Company 1641,None,0.0
6,Company 1361,None,0.0
7,Company 1321,None,0.0
8,Company 1471,None,0.0
9,Company 961,None,0.0


There seems to be some null values in the data set's Pct_Women_in_Management field, so we'll filter out those null values. 
We'll also see how many companies are there that do not have data in the Pct_Women_in_Management field, as it also indicates the state of our data. 

In [37]:
qry = """    
SELECT Company, Pct_Women_in_Management, Pct_Women_on_Board
FROM esg_data
WHERE Fiscal_Year = 2020 AND Pct_Women_in_Management IS NOT NULL
ORDER BY Pct_Women_in_Management ASC, Pct_Women_on_Board ASC
LIMIT 10;
"""

pd.read_sql_query(qry, con = connection)

,Company,Pct_Women_in_Management,Pct_Women_on_Board
0,Company 919,0.0000,0.056576
1,Company 230,0.0000,0.152525
2,Company 547,0.0087,0.064567
3,Company 972,0.0089,0.238563
4,Company 1036,0.0120,0.111060
5,Company 1045,0.0148,0.000000
6,Company 1381,0.0177,0.053032
7,Company 1090,0.0186,0.062888
8,Company 1026,0.0214,0.071460
9,Company 1173,0.0230,0.292223


In [38]:
qry = """    
SELECT COUNT(DISTINCT Company)
FROM esg_data
WHERE Fiscal_Year = 2020 AND Pct_Women_in_Management IS NULL;
"""

pd.read_sql_query(qry, con = connection)

,COUNT(DISTINCT Company)
0,803


In [43]:
qry = """    
SELECT COUNT(DISTINCT Company) *100 / (SELECT COUNT(DISTINCT Company) FROM esg_data) AS Pct_Comp_wo_WoBoard_data
FROM esg_data
WHERE Fiscal_Year = 2020 AND Pct_Women_in_Management IS NULL;
"""

pd.read_sql_query(qry, con = connection)

,Pct_Comp_wo_WoBoard_data
0,48


So we have 803 out of 1656 companies that do not have information on the % if women on their board in 2020 alone, or 48%. 

### Environment

Coming to the Environmental aspect, we are going to examine first the top and bottom industries when it comes to CO2 emissions, energy consumption and waste across 5 years then in 2020.

#### CO2 Emissions

In [53]:
qry = """    
SELECT Industry, ROUND(AVG(CO2_Emissions),1) AS Avg_CO2_Emissions
FROM esg_data
GROUP BY Industry
ORDER BY Avg_CO2_Emissions DESC
LIMIT 5;
"""

pd.read_sql_query(qry, con = connection)

,Industry,Avg_CO2_Emissions
0,Utilities,49550.6
1,Energy,27801.0
2,Materials,17053.9
3,Industrials,4047.8
4,Consumer Staples,2090.4


In [52]:
qry = """    
SELECT Industry, ROUND(AVG(CO2_Emissions),1) AS Avg_CO2_Emissions
FROM esg_data
GROUP BY Industry
ORDER BY Avg_CO2_Emissions ASC
LIMIT 5;
"""

pd.read_sql_query(qry, con = connection)

,Industry,Avg_CO2_Emissions
0,Real Estate,96.1
1,Technology,160.7
2,Communications,189.6
3,Health Care,244.4
4,Financials,463.9


Now in 2020:

In [54]:
qry = """    
SELECT Industry, ROUND(AVG(CO2_Emissions),1) AS Avg_CO2_Emissions
FROM esg_data
WHERE Fiscal_Year = 2020
GROUP BY Industry
ORDER BY Avg_CO2_Emissions DESC
LIMIT 5;
"""

pd.read_sql_query(qry, con = connection)

,Industry,Avg_CO2_Emissions
0,Utilities,37532.8
1,Energy,28967.5
2,Materials,16790.2
3,Industrials,2493.4
4,Consumer Staples,2014.3


In [55]:
qry = """    
SELECT Industry, ROUND(AVG(CO2_Emissions),1) AS Avg_CO2_Emissions
FROM esg_data
WHERE Fiscal_Year = 2020

GROUP BY Industry
ORDER BY Avg_CO2_Emissions ASC
LIMIT 5;
"""

pd.read_sql_query(qry, con = connection)

,Industry,Avg_CO2_Emissions
0,Financials,60.7
1,Real Estate,87.2
2,Technology,115.4
3,Communications,180.5
4,Health Care,256.6


Financials seem to manage to get out of the bottom position in terms of CO2 emission in 2020, indicating that there are some improvements. 

In [ ]:
#when done with the cursor and database, we close it
cursor.close()
connection.close()